In [ ]:
import json
import geopandas as gpd
from shapely.geometry import MultiPoint
import plotly.express as px
import random
import folium
import folium.plugins as plugins
from shapely.geometry import MultiPoint
import branca.colormap as cm
from folium.plugins import GroupedLayerControl

In [ ]:
# admin dasar
# admin_gdf = gpd.read_file('../data/SHP/Administrasi.shp')
# admin_gdf['centroid_latitude'] = admin_gdf['geometry'].centroid.y
# admin_gdf['centroid_longitude'] = admin_gdf['geometry'].centroid.x
# bintuni_dasar_gdf = admin_gdf.dissolve(by='Kabupaten').reset_index()
# bintuni_dasar_gdf

# sagu
topografi_gdf = gpd.read_file('../data/SHP/Topografi.shp')
# projected_crs = 'EPSG:3857' #Untuk Web Mercator (digunakan oleh OpenStreetMap, Google Maps, dan lainnya):
# sagu_gdf = sagu_gdf.to_crs(projected_crs)
# topografi_gdf['centroid_latitude'] = topografi_gdf['geometry'].centroid.y
# topografi_gdf['centroid_longitude'] = topografi_gdf['geometry'].centroid.x
# topografi_gdf = topografi_gdf.dissolve(by='Penggunaan').reset_index()
topografi_gdf['batas_awal']=topografi_gdf['Topografi'].apply(lambda x: x.split(' - ')[0]).astype(int)
topografi_gdf.sort_values(by='batas_awal', inplace=True)
topografi_gdf = topografi_gdf.drop(1).reset_index(drop=True) # drop 1 karena salah dari resky
topografi_kategorikal = topografi_gdf.dissolve(by='Dataran').reset_index()

topografi_gdf.geometry = topografi_gdf.simplify(tolerance=0.001)
tp_rendah = topografi_gdf[topografi_gdf['Dataran']=="Dataran Rendah"]
tp_tinggi = topografi_gdf[topografi_gdf['Dataran']=="Dataran Tinggi"]
topografi_gdf

In [ ]:
topografi_kategorikal

In [ ]:
colormap_rendah = cm.LinearColormap(
    ('#1b8515','#d4c92e'), 
    index=None,
    vmin=tp_rendah.batas_awal.min(), 
    vmax=tp_rendah.batas_awal.max(),
    caption="Ketinggian Tempat (mdpl)", 
    # max_labels=10, 
    tick_labels=tp_rendah.batas_awal.tolist()
)

colormap_tinggi = cm.LinearColormap(
    ('#d4c92e','#9d1e1e'), 
    index=None,
    vmin=tp_tinggi.batas_awal.min(), 
    vmax=tp_tinggi.batas_awal.max(),
    caption="Ketinggian Tempat (mdpl)", 
    # max_labels=10, 
    tick_labels=tp_tinggi.batas_awal.tolist()
)

# colormap_legend = cm.LinearColormap(
#     ('#1b8515','#d4c92e','#9d1e1e'), 
#     index=None,
#     vmin=tp_rendah.batas_awal.min(), 
#     vmax=tp_tinggi.batas_awal.max(),
#     caption="Ketinggian Tempat (mdpl)", 
#     # max_labels=10, 
#     tick_labels=(tp_rendah.batas_awal.tolist()+tp_tinggi.batas_awal.tolist())
# )
# colormap_legend.to_step(11)

In [ ]:
m = folium.Map([-2.033975531897878, 133.346063060061], zoom_start=9)

# fg_skalar = folium.FeatureGroup(name='Topografi skalar')
# m.add_child(fg_skalar)
# fg_kategorikal = folium.FeatureGroup(name='Topografi kategorikal')
# m.add_child(fg_kategorikal)

# ============== SKALAR ==============

# Create FeatureGroup for grouping GeoJson layers
skalar_group = folium.FeatureGroup(name="Topografi Skalar")

folium.GeoJson(
    tp_rendah,
    name="Topografi Rendah",
    style_function=lambda feature: {
        "fillColor": colormap_rendah(feature['properties']['batas_awal']),
        "color": colormap_rendah(feature['properties']['batas_awal']),
        "weight": 0.5,
        "dashArray": "5, 5",
        "fillOpacity": 0.9,
    },
    # smooth_factor=2,
).add_to(skalar_group)


folium.GeoJson(
    tp_tinggi,
    name="Topografi Tinggi",
    style_function=lambda feature: {
        "fillColor": colormap_tinggi(feature['properties']['batas_awal']),
        "color": colormap_tinggi(feature['properties']['batas_awal']),
        "weight": 0.5,
        "dashArray": "5, 5",
        "fillOpacity": 0.9,
    },
    # smooth_factor=2,
).add_to(skalar_group)

# Add the FeatureGroup to the map
skalar_group.add_to(m)

# ============== KATEGORIKAL ==============


# Create FeatureGroup for grouping GeoJson layers
kategorik_group = folium.FeatureGroup(name="Topografi Kategorikal", show=False)

# Add GeoJson layers to the FeatureGroup
folium.GeoJson(
    topografi_kategorikal[topografi_kategorikal['Dataran'] == "Dataran Rendah"],
    name="Topografi Rendah - Kategorikal",
    style_function=lambda feature: {
        "fillColor": '#1b8515',
        "color": '#092f07',
        "weight": 0.5,
        "dashArray": "5, 5",
        "fillOpacity": 0.9,
    },
    # smooth_factor=2,
).add_to(kategorik_group)

folium.GeoJson(
    topografi_kategorikal[topografi_kategorikal['Dataran'] == "Dataran Tinggi"],
    name="Topografi Tinggi - Kategorikal",
    style_function=lambda feature: {
        "fillColor": '#9d1e1e',
        "color": '#3f0909',
        "weight": 0.5,
        "dashArray": "5, 5",
        "fillOpacity": 0.9,
    },
    # smooth_factor=2,
).add_to(kategorik_group)

# Add the FeatureGroup to the map
kategorik_group.add_to(m)

# ============== AKHIR ==============

# GroupedLayerControl(
#     position="topright",
#     groups={'Peta': [skalar_group, kategorik_group]},
#     exclusive_groups=True,
#     collapsed=True,
# ).add_to(m)


# colormap_legend.add_to(m)
colormap_rendah.add_to(m)
colormap_tinggi.add_to(m)

# Add Legend HTML
legend_html = """
<div style="position: fixed; 
             bottom: 30px; right: 10px; width: 200px; height: auto; 
             border:2px solid grey; z-index:9999; font-size:14px;
             background-color:white;
             ">&nbsp; Legend <br>
             &nbsp; Dataran Tinggi &nbsp; <i class="fa fa-square" style="color:#9d1e1e"></i><br>
             &nbsp; Dataran Rendah &nbsp; <i class="fa fa-square" style="color:#1b8515"></i><br>
</div>"""
m.get_root().html.add_child(folium.Element(legend_html))

plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)

folium.TileLayer("esri-worldimagery").add_to(m)
folium.TileLayer("cartodbpositron").add_to(m)
folium.LayerControl().add_to(m)

In [ ]:
m.save('../map-html/topografi.html')